# SBIR/STTR DB Search Using UEI
---

### Data Source: https://www.sbir.gov/portfolio

---
This script searches the SBIR/STTR database for companies of interest using their UEI (Unique Entity Identification) number. If you need to search for multiple company UEIs, please refer to the script labeled ueiSearch.ipynb.

If a company is found in the database, two CSV files will be generated:

- `company_info_sbirsttr_db.csv`: This file contains information from the company profile, including address, number of employees, website, and more. Additional data points from each company profile can be pulled, so feel free to customize the script according to your needs.

- `sbirsttr_funding.csv`: This file includes the SBIR/STTR funding records associated with each company. It contains data such as start date, end date, funding amount, contract number, solicitation number, and more. You can also customize which data points are included in this file.


Please note that using a UEI for searching is not the only method available. Due to inconsistencies in naming across different sources, using a common identifier like a UEI or DUNS number often provides more reliable results. While you can perform a broader search using company names, this approach may require more effort in data cleaning.

If you have any questions, feel free to contact me via email at nour1786@umd.edu.


**Nour Ali Ahmed**

**UMD I-Corps Program Analyst**

**Mid-Atlantic Hub Evaluation Lead**



In [43]:
import requests
from lxml import html
import pandas as pd
from datetime import datetime

In [44]:

# Function to fetch the search results and extract the link to the first result
def get_first_result_link(uei, company_pages):
    search_url = f'https://legacy.www.sbir.gov/sbirsearch/firm/all?firm=&uei={uei}&city=&zip=&page=1'
    response = requests.get(search_url)

    if response.status_code == 200:
        tree = html.fromstring(response.content)

        # Extract the first result link
        links = tree.xpath('//table//tr/td/a/@href')

        if links:
            company_pages[uei] = 'https://legacy.www.sbir.gov' + links[0]
        else:
            print(f'{uei} search: None Found')
    else:
        print(f"Failed to retrieve search results: {response.status_code}")
    
    # print("company_pages: " , company_pages)
    return company_pages

In [45]:
# Function to fetch detailed page and extract information
def scrape_company_profile(profile_page_url):
    response = requests.get(profile_page_url)

    if response.status_code == 200:
        tree = html.fromstring(response.content)

        name_xpath = '//h1[@class="page-header"]/text()'
        street_address_xpath = '//span[@itemprop="streetAddress"]/text()'
        city_xpath = '//span[@itemprop="addressLocality"]/text()'
        state_xpath = '//span[@itemprop="addressRegion"]/text()'
        zip_xpath = '//span[@itemprop="postalCode"]/text()'
        website_xpath = '//a[@title="Company Website"]/@href'
        employee_xpath = '//div[@class="row open-style"]//div[@class="col-md-4"][2]//p[strong[contains(text(), "# of Employees:")]]/text()'

        company_name = tree.xpath(name_xpath)
        street_address = tree.xpath(street_address_xpath)
        city = tree.xpath(city_xpath)
        state = tree.xpath(state_xpath)
        zip_code = tree.xpath(zip_xpath)
        website = tree.xpath(website_xpath)
        employee_count = tree.xpath(employee_xpath)

        # Clean extracted data
        company_name = company_name[0].strip() if company_name else "N/A"
        street_address = street_address[0].strip().title() if street_address else "N/A"
        city = city[0].strip().title() if city else "N/A"
        state = state[0].strip() if state else "N/A"
        zip_code = zip_code[0].strip() if zip_code else "N/A"
        website = website[0].strip() if website else "N/A"
        employee_count = employee_count[0].strip() if employee_count else "N/A"

        # Create a DataFrame for this company's data
        company_df = pd.DataFrame([{
            "Name": company_name,
            "Street Address": street_address,
            "City": city,
            "State": state,
            "Zip Code": zip_code,
            "Website": website,
            "Employee Count": employee_count,
            "SBIR Profile Link": profile_page_url
        }])

        # Extract award links
        awards_xpath = '//div[@class="firm-details-content"]//h3/a/@href'
        awards_links = tree.xpath(awards_xpath)

        # Return the DataFrame and the list of award links
        # print("company_df: " , company_df)
        # print("awards_links: " , awards_links)
        return company_df, [f'https://legacy.www.sbir.gov{link}' for link in awards_links]
    else:
        print(f"Failed to retrieve detailed page: {response.status_code}")
        return pd.DataFrame(), []


In [46]:
def format_date(date_str):
    """Convert date from YYYY-MM-DD to MM/DD/YYYY format."""
    try:
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        return date_obj.strftime('%m/%d/%Y')
    except ValueError:
        return date_str  # Return the original string if parsing fails


In [ ]:
# Function to scrape funding records from award pages
def scrape_award_page(award_url):
    response = requests.get(award_url)

    if response.status_code == 200:
        tree = html.fromstring(response.content)

        # Define the XPaths for various fields
        award_start_date_xpath = '//span[@class="open-label" and contains(text(), "Award Start Date (Proposal Award Date):")]/following-sibling::span[@class="open-description"]/text()'
        award_end_date_xpath = '//span[@class="open-label" and contains(text(), "Award End Date (Contract End Date):")]/following-sibling::span[@class="open-description"]/text()'
        # duns_num_xpath = '//div[@class="row open-style"]//span[@class="open-label" and contains(text(), "DUNS:")]/following-sibling::span[@class="open-description"]/text()'
        amount_xpath = '//div[@class="row open-style"]//span[@class="open-label" and contains(text(), "Amount:")]/following-sibling::span[@class="open-description"]/text()'
        phase_xpath = '//div[@class="row open-style"]//span[@class="open-label" and contains(text(), "Phase:")]/following-sibling::span[@class="open-description"]/text()'
        program_xpath = '//div[@class="row open-style"]//span[@class="open-label" and contains(text(), "Program:")]/following-sibling::span[@class="open-description"]/text()'
        solicitation_number_xpath = '//div[@class="row open-style"]//span[@class="open-label" and contains(text(), "Solicitation Number:")]/following-sibling::span[@class="open-description"]/text()'
        company_name_xpath = '//div[@class="sbc-name-wrapper"]/a/text()'

        # Extract data using the defined XPaths
        award_start_date = tree.xpath(award_start_date_xpath)
        award_end_date = tree.xpath(award_end_date_xpath)
        # duns_num = tree.xpath(duns_num_xpath)  # not being used
        amount = tree.xpath(amount_xpath)
        phase = tree.xpath(phase_xpath)
        program = tree.xpath(program_xpath)
        solicitation_number = tree.xpath(solicitation_number_xpath)
        company_name = tree.xpath(company_name_xpath)

        # Format dates
        formatted_award_start_date = format_date(award_start_date[0].strip()) if award_start_date else "N/A"
        formatted_award_end_date = format_date(award_end_date[0].strip()) if award_end_date else "N/A"

        # Return the scraped data as a dictionary including the award URL
        return {
            "Company Name": company_name[0].strip() if company_name else "N/A",
            "Start Date": formatted_award_start_date,
            "End Date": formatted_award_end_date,
            "Funding Amount": amount[0].strip() if amount else "N/A",
            "Phase": phase[0].strip() if phase else "N/A",
            "Program": program[0].strip() if program else "N/A",
            "Solicitation Number": solicitation_number[0].strip() if solicitation_number else "N/A",
            "Source Link" : award_url
        }
    else:
        print(f"Failed to retrieve award page: {response.status_code}")
        return None

In [ ]:

## TO- change to list pulled from input file

df = pd.read_csv("UEI_test_list.csv")
#uei_list = df.to_list()
#uei_list = df[df["num_uei"]].to_list()
uei_list = df['num_uei'].tolist()

company_info_df = pd.DataFrame(columns=[
    "Name", "Street Address", "City", "State", "Zip Code", "Website", "Employee Count" , "SBIR Profile Link"
])
company_pages = {}
funding_records = []


# Iterate through the list of UEIs and fetch the corresponding page links
for uei in uei_list:
    company_pages = get_first_result_link(uei, company_pages)
    profile_page_url = company_pages.get(uei)
    if profile_page_url:
        company_df, award_links = scrape_company_profile(profile_page_url)
        company_info_df = pd.concat([company_info_df, company_df], ignore_index=True)
        for award_link in award_links:
            record = scrape_award_page(award_link)
            if record:
                funding_records.append(record)

# Save company information to a CSV file
company_info_df.to_csv('company_info_sbirsttr_db.csv', index=False)

# Save funding records to a CSV file
df_funding = pd.DataFrame(funding_records)
df_funding.to_csv('sbirsttr_funding.csv', index=False)

YPL3PSFKSJ49 search: None Found
GXVCMTLKZP96 search: None Found
VGLLXNMDNHQ5 search: None Found
KF5LQ1X8QF87 search: None Found
EVVCNB5YZUS3 search: None Found
TCT3X4JFLTP8 search: None Found
P25AZ6WDJ589 search: None Found
MHJUNMMNSN47 search: None Found
K6U9KE5XD7B1 search: None Found
WSHDCNQPGPC9 search: None Found
XP7KJNLQRFP3 search: None Found
U1NSNPBC3J14 search: None Found
HV1DC9MV4F17 search: None Found
KW2NMJ1EGY69 search: None Found
UYWZM8MHL2W7 search: None Found
YBJNM8JECF46 search: None Found
RQVUQQ9T9MT6 search: None Found
SXRRDFTN2547 search: None Found
FCKARMCJLC51 search: None Found
U54CHNEY9DB8 search: None Found
CXHKZ4HG79W4 search: None Found
UYGVDZDSW9Q3 search: None Found
VGQWB5E7F3K3 search: None Found
UJ6KWR81THW3 search: None Found
T81CKJG4CM38 search: None Found
KDXUA129N265 search: None Found
T9Q7HEENL936 search: None Found
J9AEEM6ZNVU6 search: None Found
S324XC4V3N25 search: None Found
RCWDXBK44D73 search: None Found
T37JWJDD79T4 search: None Found
WKDMT9PH

KeyboardInterrupt: 